In [1]:
lines = sc.parallelize(["hello world", "hi"])
words = lines.flatMap(lambda line: line.split(" "))
words.first()

'hello'

In [2]:
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
nums_sum = nums.reduce(lambda x,y: x+y)
print(nums_sum)

55


In [3]:
print(nums.take(3))
print(nums.top(3))
print(nums.mean())
print(nums.variance())

[1, 2, 3]
[10, 9, 8]
5.5
8.25


In [4]:
from pyspark import StorageLevel
result = nums.map(lambda x: x * x)
# result.persist(StorageLevel.DISK_ONLY) 
result.persist()
print(result.count())
print(result.collect())

10
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [5]:
isinstance(StorageLevel.DISK_ONLY, StorageLevel)

True

# Key Value operation

In [6]:
pair_rdd = sc.parallelize([(1, 2), (3, 4), (3,6), (2,2)])

In [7]:
pair_rdd.reduceByKey(lambda x, y: x+y).collect()

[(1, 2), (2, 2), (3, 10)]

In [10]:
for k, v in pair_rdd.groupByKey().collect():
    print(k, [xx for xx in v])

1 [2]
2 [2]
3 [4, 6]


In [11]:
pair_rdd.mapValues(lambda x: x+1).collect()

[(1, 3), (3, 5), (3, 7), (2, 3)]

In [12]:
pair_rdd.keys().collect()

[1, 3, 3, 2]

In [13]:
pair_rdd.values().collect()

[2, 4, 6, 2]

In [14]:
pair_rdd.sortByKey().collect()

[(1, 2), (2, 2), (3, 4), (3, 6)]

In [15]:
pair_rdd2 = sc.parallelize([(3, 20)]) 
pair_rdd2.collect()

[(3, 20)]

In [16]:
pair_rdd.collect()

[(1, 2), (3, 4), (3, 6), (2, 2)]

In [17]:
pair_rdd.subtractByKey(pair_rdd2).collect() #remove key 3

[(1, 2), (2, 2)]

In [18]:
pair_rdd.join(pair_rdd2).collect()

[(3, (4, 20)), (3, (6, 20))]

In [19]:
pair_rdd.leftOuterJoin(pair_rdd2).collect()

[(1, (2, None)), (2, (2, None)), (3, (4, 20)), (3, (6, 20))]

In [20]:
pair_rdd.rightOuterJoin(pair_rdd2).collect()

[(3, (4, 20)), (3, (6, 20))]

In [21]:
pair_rdd.cogroup(pair_rdd2).collect()

[(1,
  (<pyspark.resultiterable.ResultIterable at 0x11030ca58>,
   <pyspark.resultiterable.ResultIterable at 0x11030cbe0>)),
 (2,
  (<pyspark.resultiterable.ResultIterable at 0x11030ca90>,
   <pyspark.resultiterable.ResultIterable at 0x11030cba8>)),
 (3,
  (<pyspark.resultiterable.ResultIterable at 0x11030cda0>,
   <pyspark.resultiterable.ResultIterable at 0x11030c9e8>))]

In [24]:
pair_rdd.reduceByKey(lambda x,y: x+y).collect()

[(1, 2), (2, 2), (3, 10)]

# 数据分区

In [25]:
print(pair_rdd.partitioner)

None


In [26]:
p2 = pair_rdd.partitionBy(2).persist()

In [27]:
p2.partitioner.numPartitions

2

# debug and optimization

In [28]:
pair_rdd.toDebugString()

b'(4) ParallelCollectionRDD[12] at parallelize at PythonRDD.scala:475 []'

In [29]:
pair_rdd2.toDebugString()

b'(4) ParallelCollectionRDD[38] at parallelize at PythonRDD.scala:475 []'